In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/Users/aa/Desktop/coding/p3ai/t2final/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading AI model...


Loading weights: 100%|██████████| 76/76 [00:00<00:00, 1465.99it/s, Materializing param=transformer.wte.weight]            
GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [3]:
# Simple example
text = "The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories"
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate {steps} words, one at a time so each step is a token
current_text = text
for step in range(40): ##num of steps (tokens)
    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")
    
    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")
    
    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits
    
    # Get the predictions for the NEXT token
    next_token_logits = predictions[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=0)
    
    # Get top k (5) predictions
    k=5;
    top_probs, top_indices = torch.topk(next_token_probs, k) ##the int in the second param is k
    
    print("Top 5 next word predictions:")
    for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
        word = tokenizer.decode([idx])
        print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")
    
    top_probs = top_probs/top_probs.sum(); ##normalize to 1
    choice = torch.multinomial(top_probs, 1).item() ##pick a random sample of the top k tokens
    next_token_id = top_indices[choice].item() #use the randomly selected word instead to avoid boring responses
    
    # Use the most likely word
    ##next_token_id = top_indices[0] ##this just picks the first word
    next_word = tokenizer.decode([next_token_id])
    current_text += next_word
    
    print(f"✓ Chosen: '{next_word}'")
    print(f"New text: '{current_text}'\n")

print(f"\nFinal generated text: '{current_text}'")


Starting text: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories'

Generating word by word...

--- Step 1 ---
Current: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories'
Top 5 next word predictions:
  1. '.' (51.5% confident)
  2. ' of' (14.1% confident)
  3. ' about' (6.7% confident)
  4. ' and' (5.7% confident)
  5. ',' (3.9% confident)
✓ Chosen: '.'
New text: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories.'

--- Step 2 ---
Current: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a so

In [4]:
# Simple example
text = "The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories"
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate 5 words, one at a time
current_text = text
for step in range(50): ##num of steps
    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")
    
    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")
    
    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits
    
    #get predictions for the next token
    next_token_logits = predictions[0, -1, :]
    
    #only choose the single highest probability word
    next_token_id = torch.argmax(next_token_logits, dim=-1)

    #for debugging, print probabilities
    next_token_probs = torch.softmax(next_token_logits, dim=0)
    top_prob = next_token_probs[next_token_id].item()
    
    #perform the change to the current text by adding the next predicted token onto it
    next_word = tokenizer.decode([next_token_id])
    current_text += next_word
    
    #end by printing new text and the word with the confidence
    print(f"✓ Chosen: '{next_word}' ({top_prob*100:.1f}% confident)")
    print(f"New text: '{current_text}'\n")

print(f"\nFinal generated text: '{current_text}'")


Starting text: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories'

Generating word by word...

--- Step 1 ---
Current: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories'
✓ Chosen: '.' (51.5% confident)
New text: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories.'

--- Step 2 ---
Current: 'The moon was shining bright over the quiet village, casting long silver shadows across the empty streets as a soft wind moved through the trees and the night felt calm but full of unspoken stories.'
✓ Chosen: '
' (37.1% confident)
New text: 'The moon was shining br